<a href="https://colab.research.google.com/github/GemmaRagadini/Pokemon_AIF_24_25/blob/dev/pokemon-vgc-engine-master/example/Notebook_project_Pokemon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pokemon_Battle PokeBob team**

The project focuses on the track proposed in the course, specifically the section related to competitions. Our team selected the task involving the simulation of a Pokémon battle between two teams, each composed of three Pokémon. The battle consists of three matches, with the first player to knock out all three Pokémon of the opposing player declared the winner of the match. The battle is considered concluded when a player wins at least two out of three matches.

The objective of the project was to develop an AI agent capable of defeating a random player. This random player selects its Pokémon moves arbitrarily without any strategic logic. Both players operate under the same conditions, with their teams assigned randomly, ensuring that any advantages or disadvantages are also determined by chance
In the following sections, we will explain our approach to solving the task and outline the methodology we adopted, along with the results we obtained.  (capire come far andare i codici su colab)


In [1]:
# Clone the entire repo.
!git clone https://github.com/GemmaRagadini/Pokemon_AIF_24_25.git
%cd Pokemon_AIF_24_25/pokemon-vgc-engine-master/example
!ls

Cloning into 'Pokemon_AIF_24_25'...
remote: Enumerating objects: 1858, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 1858 (delta 190), reused 128 (delta 77), pack-reused 1614 (from 1)
Receiving objects: 100% (1858/1858), 21.61 MiB | 28.05 MiB/s, done.
Resolving deltas: 100% (412/412), done.
/content/Pokemon_AIF_24_25/pokemon-vgc-engine-master/example
Example_BattleEcosystem.py	  Example_PkmBattleEnv.py
Example_ChampionshipEcosystem.py  Example_RemoteChampionshipEcosystem.py
Example_Competitor.py		  Example_RemoteCompetitor.py
Example_GeneratePkmRoster.py	  Example_Teambuild.py
Example_Match.py		  Example_TreeChampionship.py


In [2]:

#from Example_BattleEcosystem import Tournament, main
from Example_BattleEcosystem.py import main

main()

ModuleNotFoundError: No module named 'vgc'

**Related Work**

For this project, we decided to explore some existing approaches related to the concepts studied during the course, as well as to develop a custom approach of our own. These approaches were evaluated through a tournament, the details of which will be provided in the corresponding section.

The related work was derived from the course slides and the accompanying textbook, Artificial Intelligence Fundamentals. Specifically, we focused on the section related to game theory (Chapter 6 of the textbook) and examined various approaches to the Minimax algorithm and its variations. This included the implementation of alpha-beta pruning and heuristics such as the killer move heuristic.

**Methodology**

To achieve our goal, we decided to implement several algorithms discussed in the related work. Initially, we focused our attention on various implementations of the Minimax algorithm. In the following sections, we will provide a detailed explanation of each algorithm we implemented.

**Minimax**

The first implementation is the one obout a simple minimax with a simple evaluation function called game_eval. Here below there is our implementation.


In [ ]:


def game_state_eval(s: GameState, depth):
    mine = s.teams[0].active
    opp = s.teams[1].active
    return mine.hp / mine.max_hp - 3 * opp.hp / opp.max_hp - 0.3 * depth


def n_fainted(t: PkmTeam):
    fainted = 0
    fainted += t.active.hp == 0
    if len(t.party) > 0:
        fainted += t.party[0].hp == 0
    if len(t.party) > 1:
        fainted += t.party[1].hp == 0
    return fainted




class MyMinimax(BattlePolicy):

    def __init__(self, max_depth: int = 4):
        self.max_depth = max_depth
        self.name = "Minimax"

    def minimax(self, g, depth, is_maximizing_player):
        """
        Classic Minimax with basic evaluation function.

        :param g: current game state.
        :param depth: the depth of the research.
        :param is_maximizing_player: True if the player is a maximaxer or False if it is a minimazer.
        :return: (valutazione, azione migliore)
        """
        if depth == 0:
            # the evaluation function is the basic evaluation function that evaluates the Hp of the pokemons.
            return game_state_eval(g, depth), None

        if is_maximizing_player:
            max_eval = float('-inf')
            best_action = None
            for i in range(DEFAULT_N_ACTIONS):
                g_copy = deepcopy(g)
                s, _, _, _, _ = g_copy.step([i, 99])  # the enemy does not do a correct action the state does not change
                if n_fainted(s[0].teams[0]) > n_fainted(g.teams[0]):
                    continue # ignores the state where our pokemon loose.
                eval_score, _ = self.minimax(s[0], depth - 1, False)
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_action = i
            return max_eval, best_action

        else:  # part of the enemy where he tries to minimize
            min_eval = float('inf')
            best_action = None
            for j in range(DEFAULT_N_ACTIONS):
                g_copy = deepcopy(g)
                s, _, _, _, _ = g_copy.step([99, j])  # The player does not change the action (not valid action)
                # it ignores the state where the defeated pokemon of the enemy increase.
                if n_fainted(s[0].teams[1]) > n_fainted(g.teams[1]):
                    continue
                eval_score, _ = self.minimax(s[0], depth - 1, True)
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_action = j
            return min_eval, best_action

    def get_action(self, g) -> int:
        """
        bets action to do by the maximazer player.

        :param g: current game state.
        :return: bets action to do.
        """
        _, best_action = self.minimax(g, self.max_depth, True)
        return best_action if best_action is not None else 0



class Node:
    def __init__(self, state, parent, player, action=None):
        self.state = state
        self.parent = parent
        self.player = player
        self.action = action
        self.children = []
        self.visits = 0
        self.value = 0

    def is_fully_expanded(self):
        return len(self.get_untried_actions()) == 0

    def get_untried_actions(self):
        # Ottiene le azioni possibili dallo stato
        return [i for i in range(DEFAULT_N_ACTIONS)]


In the cell above, we present our first implementation of the Minimax policy, which employs a basic evaluation function called game_state_eval. This function is designed to:

Encourage states where the player’s active Pokémon (mine) has higher HP relative to its maximum HP.
Penalize states where the opponent’s active Pokémon (opp) has high HP.
Add a penalty proportional to the search depth to prioritize faster victories.
Although this evaluation function is rudimentary, our results demonstrate that it provides a balanced implementation. However, it is not the most effective approach we encountered.

The Minimax implementation is straightforward, comprising a section for the maximizer player and another for the minimizer. The maximizer aims to transition to states where its Pokémon are healthier than the opponent’s Pokémon, while the minimizer seeks to reduce this advantage. Each state is evaluated recursively.

To support these computations, the algorithm uses the n_fainted function, which counts the number of fainted (knocked-out) Pokémon. Additionally, the algorithm determines the next action from the maximizer player’s perspective, as implemented in the get_action method.

Another critical function used is the step function, which simulates actions to predict how the game state evolves. This function works in conjunction with the n_fainted function to enhance decision-making.

The default values for the search depth and weights in the evaluation function were determined empirically. Various configurations were tested, and the ones used here were found to deliver the best performance according to our evaluation metrics.

The Node class, shown at the end of the cell, represents a node in the Minimax tree. It includes several fields to facilitate tree exploration, such as parent, children, value, and state.

Additionally, the class provides two key methods:

fully_expand: Checks whether the node has been fully explored.
get_untried_actions: Retrieves the set of possible actions that can still be taken from the current node.
These fields and methods are critical for efficiently navigating and expanding the Minimax tree during the decision-making process.





**Minimax with Alpha-Beta Pruning and Killer Move Heuristic**

Our second implementation extends the basic Minimax algorithm by incorporating alpha-beta pruning and the killer move heuristic. This implementation was developed to enhance both the performance and efficiency of the Minimax algorithm described above.

The addition of alpha-beta pruning allows the algorithm to eliminate branches in the search tree that cannot influence the final decision, significantly reducing the number of nodes explored. Meanwhile, the killer move heuristic prioritizes moves that are likely to be effective, further optimizing the decision-making process by focusing on promising actions.

The combined use of these techniques aims to not only improve the accuracy of the algorithm but also speed up its execution, enabling faster and more effective decision-making.

In the cells below, we present our implementation of this enhanced Minimax algorithm along with a detailed explanation of how it operates.

In [ ]:
class MyMinimaxWithAlphaBetaKiller(BattlePolicy):

    def __init__(self, max_depth: int = 5):
        self.max_depth = max_depth
        self.name = "Minimax with pruning alpha beta killer"
        self.killer_moves = {depth: [] for depth in range(max_depth + 1)}  # Memorizza le killer moves per profondità

    def minimax(self, g, depth, alpha, beta, is_maximizing_player):
        if depth == 0:
            return evalFunctions.game_state_eval(g, depth), None

        if is_maximizing_player:
            max_eval = float('-inf')
            best_action = None

            # Ottieni le azioni disponibili
            moves = list(range(DEFAULT_N_ACTIONS))

            # Prioritizza le killer moves
            killer_moves = self.killer_moves.get(depth, [])
            moves = sorted(moves, key=lambda move: move in killer_moves, reverse=True)

            for i in moves:
                g_copy = deepcopy(g)
                s, _, _, _, _ = g_copy.step([i, 99])
                if evalFunctions.n_fainted(s[0].teams[0]) > evalFunctions.n_fainted(g.teams[0]):
                    continue

                eval_score, _ = self.minimax(s[0], depth - 1, alpha, beta, False)
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_action = i

                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    # Aggiorna le killer moves
                    if i not in self.killer_moves[depth]:
                        self.killer_moves[depth].append(i)
                        if len(self.killer_moves[depth]) > 2:
                            self.killer_moves[depth].pop(0)
                    break
            return max_eval, best_action

        else:
            min_eval = float('inf')
            best_action = None

            # Ottieni le azioni disponibili
            moves = list(range(DEFAULT_N_ACTIONS))

            # Prioritizza le killer moves
            killer_moves = self.killer_moves.get(depth, [])
            moves = sorted(moves, key=lambda move: move in killer_moves, reverse=True)

            for j in moves:
                g_copy = deepcopy(g)
                s, _, _, _, _ = g_copy.step([99, j])
                if evalFunctions.n_fainted(s[0].teams[1]) > evalFunctions.n_fainted(g.teams[1]):
                    continue

                eval_score, _ = self.minimax(s[0], depth - 1, alpha, beta, True)
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_action = j

                beta = min(beta, eval_score)
                if beta <= alpha:
                    # Aggiorna le killer moves
                    if j not in self.killer_moves[depth]:
                        self.killer_moves[depth].append(j)
                        if len(self.killer_moves[depth]) > 2:
                            self.killer_moves[depth].pop(0)
                    break
            return min_eval, best_action

    def get_action(self, g) -> int:
        _, best_action = self.minimax(g, self.max_depth, float('-inf'), float('inf'), True)
        return best_action if best_action is not None else 0

This section details the implementation of an advanced Minimax algorithm enhanced with Alpha-Beta pruning and the Killer Moves heuristic. The algorithm optimizes decision-making in competitive game scenarios by efficiently evaluating possible actions and pruning less relevant game tree branches.

Overview

The Minimax algorithm aims to find the best possible action for a player in a turn-based game by exploring potential future states. This implementation improves the basic Minimax algorithm with two key techniques:

Alpha-Beta Pruning: Reduces the number of nodes evaluated by eliminating branches that cannot affect the final decision.

Killer Moves Heuristic: Prioritizes actions ("killer moves") that have previously led to significant cutoffs, increasing the likelihood of early pruning.

Attributes

max_depth: Specifies the maximum depth of the search tree.

name: Identifies the policy as "Minimax with pruning alpha beta killer."

killer_moves: A dictionary mapping depths to lists of "killer moves." Each depth can store up to two killer moves that caused Beta cutoffs.

(da finire)

**Custom Evaluation Function**

We decided to implement an other evaluation function that differs in the approch respect to the one described above. This new evaluation function is a more aggressive evaluation function in fact take into consideration only the power and the effectivness of a move against the opponent. The aim of this new evaluation function was to create a more aggresive agent so the battles could and in more rapid way. *The result obtained are illustrated and commented in the Evaluation section*.

**Custom Policy**

The third and last approch was the one about a our custom Policy. This policy differ form the other that has been implemented

Descrizione della custom Policy

**Evaluation**

We implemented several approaches and, in order to determine which policy performed best, we decided to have each agent battle against the Random agent. This allowed us to demonstrate that each policy outperforms the Random agent.

For the second evaluation, we organized a tournament involving all the players:

Random Player

Minimax Player

Minimax with Alpha-Beta Pruning and Killer Move Heuristic Player

Custom Policy Player

Each match consisted of 10 battles, and the winner was determined by the player who won the most matches, thus achieving the best win rate.

The results of each individual battle, as well as the overall winner of the tournament, are reported in the cells below.

First Battle: Minimax vs Random Player: the first result is when the two player have always a different team and the second when they have always the same team

Piccolo commento sui risultati

Second Battle: Minimax with Alpha-Beta Pruning and Killer Move Heuristic Player vs Random Player. The first result is when the two player have always a different team and the second when they have always the same team

piccolo commento sui risultati

Third Battle : Custom Player vs Random Player. The first result is when the two player have always a different team and the second when they have always the same team

piccolo commento sul risultato

Tuornament between all the player involved. The first result is when the two player have always a different team and the second when they have always the same team

Piccolo commento sul risultato

**Conclusion**

The result from the the simulations of the battle was allineated with what we thought. In fact every player with a policy different from the random one was able to defeat the random playeer, not always with outstanding results but they culd beat the random player. Our new evaluation function apllied to the minimax and to the other minimax with the alpha beta pruning and killer move heuristic function aimed to obtain a more aggressive player more focused on the attack fase and on the power of the moves in the roster. At the beginning we thougth that this could be the better approch and that battle would be ended in a very fast way. This was partialy true beacuse it was only partially more faster but the win rate was not as good as we fought maybe a more conservative approch wuolb be better. The best out of the 4 Player was the one with the custom policy. This because the custom policy takes into consideration variuos aspect of the game for example take into consideration when to do a switch between our pokemon and with whom to do this switch, so the policy was aggressive becauese aimed to search the best and powerful move but take into account also the matchup with the pokemon of the opponent, giving to the Custom policy a better knowledge of the state of the game (da rivedere)

**Appendix**
(se vogliamo metterci qualcosa ma vediamo dopo)